# Mendapatkan link artikel dari dynamic page

In [120]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd
from lxml import etree

In [121]:
driver = webdriver.Chrome()  
driver.maximize_window() 

In [122]:
driver.get("https://www.babycenter.com/getting-pregnant/how-to-get-pregnant")  

In [123]:
doc=driver.page_source
print(doc)

<html lang="en" class=" " style=""><head style=""><script id="_mN_dys_1679815636896164832" src="https://contextual.media.net/sr/1045354880/pfcmdynet.js?cb=window._mNDetails.initAd&amp;&amp;gdpr=0&amp;uspenf=2&amp;cid=8CUBF27H4&amp;cpcd=56Bc8knhNhynXal1V-6lFA%3D%3D&amp;crid=241584837&amp;size=1022x520&amp;cc=ID&amp;https=1&amp;vif=1&amp;requrl=https%3A%2F%2Fwww.babycenter.com%2Fgetting-pregnant%2Fhow-to-get-pregnant&amp;nse=5&amp;vi=1679815636896164832&amp;lw=1&amp;ugd=4&amp;sff=0&amp;kttle=How%20to%20Get%20Pregnant&amp;tsce=L84&amp;pgid=p1594295254t202303260727&amp;nb=1&amp;allsc=JK&amp;tcf_cmp=1" async=""></script><script type="text/javascript" async="" src="https://cdn.krxd.net/controltag/spgdj7g8u.js"></script><script id="_mN_dys_1679815636935950493" src="https://contextual.media.net/sr/1045354880/pfcmdynet.js?cb=window._mNDetails.initAd&amp;&amp;gdpr=0&amp;uspenf=2&amp;cid=8CUBF27H4&amp;cpcd=56Bc8knhNhynXal1V-6lFA%3D%3D&amp;crid=161944584&amp;size=1022x520&amp;cc=ID&amp;https=1&amp

In [124]:
with open('how_to_get_pregnant.html','w') as f:
    f.write(doc)

In [152]:
HTMLFile = open("how_to_get_pregnant.html", "r")
html_text = HTMLFile.read()
HTMLFile.close()
soup=BeautifulSoup(html_text,'lxml')

In [153]:
df = pd.DataFrame(columns=['category','title','link'])

articles=soup.find_all('div',class_='sc-7ac25d38-0 fBQhKS promoItem')
for article in articles:
    title=article.find('div',class_='promoTitle').text
    link=article.find('a',class_='hoverTextOnly')['href']
    link='https://www.babycenter.com'+link
    df.loc[len(df)] = ['how to get pregnant',title,link]

In [154]:
df=df.drop_duplicates()
df.head()

,category,title,link
0,how to get pregnant,Does taking fish oil for fertility really help?,https://www.babycenter.com/getting-pregnant/ho...
1,how to get pregnant,Fertility foods: What to eat when trying to co...,https://www.babycenter.com/getting-pregnant/ho...
2,how to get pregnant,How much folic acid should I take while trying...,https://www.babycenter.com/getting-pregnant/ho...
3,how to get pregnant,Caffeine: Does it affect your fertility?,https://www.babycenter.com/getting-pregnant/ho...
4,how to get pregnant,Can soy isoflavones help me get pregnant?,https://www.babycenter.com/getting-pregnant/ho...


# Mendonwload artikel html

In [133]:
n=0
for link in df.link:
    try:
        html_text=requests.get(link).text
        with open('how_to_get_pregnant'+str(n)+'.html','w') as f:
            f.write(html_text)
        print('success '+str(n))
    except:
        print('how_to_get_pregnant'+str(n)+'.html failed')
    n+=1

success 0
success 1
success 2
success 3
success 4
success 5
success 6
success 7
success 8
success 9
success 10
success 11
success 12
success 13
success 14
success 15
success 16
success 17
success 18
success 19
success 20
success 21
success 22
success 23
success 24
how_to_get_pregnant25.html failed
how_to_get_pregnant26.html failed
success 27
how_to_get_pregnant28.html failed
success 29
success 30
success 31
success 32
success 33
success 34
success 35
success 36
success 37
success 38
success 39
success 40
success 41
success 42


In [136]:
linklist=[25,26,28]
for i in linklist:
    df.link[i]=df.link[i].replace('https://www.babycenter.comhttps','https')

In [137]:
linklist=[25,26,28]
for link in linklist:
    try:
        html_text=requests.get(df.link[link]).text
        with open('how_to_get_pregnant'+str(link)+'.html','w') as f:
            f.write(html_text)
        print('success '+str(link))
    except:
        print('how_to_get_pregnant'+str(link)+'.html failed')

success 25
success 26
success 28


# Mendapatkan text artikel

In [155]:
df.title[0]

'Does taking fish oil for fertility really help?'

In [164]:
dfArticle = pd.DataFrame(columns=['article'])

for i in range(0,len(df)):
    HTMLFile = open(f"how_to_get_pregnant{i}.html", "r")
    html_text = HTMLFile.read()
    HTMLFile.close()
    soup=BeautifulSoup(html_text,'lxml')
    dom = etree.HTML(str(soup))
    article=''
    article+=df.title[i]+'\n'
    for i in dom.xpath("//div[contains(@class,'authoredContentBulletedLists')]//text()"):
        if i.getparent().tag=='p':
            article+=i
        elif i.getparent().tag=='h2':
            article+='\n'+i
        elif i.getparent().tag=='div':
            article+='\n'+i
        else:
            article+=i
    dfArticle.loc[len(dfArticle)] = [article]

In [165]:
empty=[]
for i in range(0,len(df)):
    if dfArticle.article[i]=='':
        empty+=[i]
print(empty)

[]


In [1]:
df = df.join(dfArticle)

In [163]:
df.to_csv('result.csv',encoding='utf-8-sig')